In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, precision_score, confusion_matrix, log_loss
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.feature_selection import SelectKBest, SelectFromModel, RFE, RFECV, SelectPercentile, SelectFpr, SelectFdr, SelectFwe
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
import lifelines as ll
import matplotlib.pyplot as plt
import pickle
import sys
sys.path.append('/odinn/users/thjodbjorge/Python_functions/')
import Predict_functions as pf
from Feature_selection import stepwise_selection
from Calculate_score import calculate_metrics

In [ ]:
# raw_data = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/raw_with_info.csv',index_col = 'Barcode2d' )
probe_info = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/probe_info.csv', index_col = 'SeqId')

pn_info = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/pn_info_Mor/pn_info_Mor_event.csv',index_col = 'Barcode2d' )
probes_to_skip = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/probes_to_skip.txt')['probe']


folder = '/odinn/users/thjodbjorge/Proteomics/Mortality2/'
org_folder = 'Traditional_risk_factors/'
pred_folder = 'Traditional_risk_factors_20210317/'
feat_folder = 'Features2/'

endpoints = ['death']
# endpoints = ['death','Cdeath','Gdeath','Ideath','Jdeath','Otherdeath']
# event_date = event_date_death
time_to_event = pn_info.time_to_death
no_event_before = pn_info.no_death_before
for endpoint in endpoints:
    if endpoint == 'death':
        use_event = pn_info.event_death
        print(use_event.sum())
    elif endpoint == 'Cdeath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'C')
        print(use_event.sum())
    elif endpoint == 'Gdeath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'G')
        print(use_event.sum())
    elif endpoint == 'Ideath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'I')
        print(use_event.sum())
    elif endpoint == 'Jdeath':
        use_event = pn_info.event_death & (pn_info.ICD_group == 'J')
        print(use_event.sum())
    elif endpoint == 'Otherdeath':
        use_event = pn_info.event_death & (~(pn_info.ICD_group == 'C')&~(pn_info.ICD_group == 'G')&~(pn_info.ICD_group == 'I')&~(pn_info.ICD_group == 'J'))
        print(use_event.sum())


y = []
for i in range(1,19):
    y.append(use_event & (time_to_event <= i))

In [ ]:
pn_df = pd.read_csv('/odinn/users/thjodbjorge/Proteomics/Data/qt_pheno_data.csv')
# pn_df['Time_of_plasma_collection_2'] = pd.to_datetime(pn_df.Time_of_plasma_collection_2)
pn_df.drop('Time_of_plasma_collection_2',inplace=True,axis=1)
qt_col = pn_df.columns
w1y_qt_col = qt_col[['w1y' in c for c in qt_col]]
# Maybe I should normalize all of this because I know nothing about these variables
index = pn_info.index
pn_info = pn_info.merge(pn_df, how = 'left', left_on = 'PN', right_on = 'PN',suffixes = [None,'_y'])
pn_info.index = index

extra_feat = ['Stroke','Asthma','HDL','TC','SBP']
extra_feat2 = ['Asthma','HDL','TC','SBP']

In [ ]:
kf = KFold(n_splits=10, random_state=10, shuffle=False) 
I_train_main, I_test_main = train_test_split(pn_info.index, train_size=0.7, random_state = 10)
I_train_main = pn_info.index
# I_val_main, I_test_main = train_test_split(I_test_main, train_size=0.5, random_state = 10)



file = open(folder+"{}_keep_samples.pkl".format('Mor'),'rb')
keep_samples_dict = pickle.load(file)

# print(keep_samples_dict.keys())
# keep_samples_keys = ['Old_18105', 'Old_60105', 'Old_6080','Old_18105_C', 'Old_18105_I', 'Old_18105_J', 'Old_18105_G','Old_18105_Other']
# keep_samples_keys = ['Old_women_18105']
keep_samples_keys = ['Old_18105']
# K = [9]
# K = [4]
# K =[14]
k =4
for dataset in keep_samples_keys:
    print(dataset)
    keep_samples = keep_samples_dict[dataset]

    I_train = I_train_main.intersection(keep_samples)#.intersection(have_prs)
    I_test = I_test_main.intersection(keep_samples)#.intersection(have_prs)

    print('Training set: {}, MI within 15: {}, 10: {}, 5: {}, 2: {}'.format(len(I_train),y[14][I_train].sum(),y[9][I_train].sum(),y[4][I_train].sum(),y[1][I_train].sum()))
    print('Test set: {}, MI within 15: {}, 10: {}, 5: {}, 2: {}'.format(len(I_test),y[14][I_test].sum(),y[9][I_test].sum(),y[4][I_test].sum(),y[1][I_test].sum()))

        # ### Select data and normalize

#     X = np.log(raw_data.iloc[:,16:].drop(probes_to_skip,axis=1))

#     all_protein = X.columns
    X = pd.DataFrame(index=pn_info.index)
    X['sex'] = pn_info[['sex']].values-1
    X['age'] = pn_info[['Age_at_sample_collection_2']].values

    X['age2'] = X['age']**2
#     X['age3'] = X['age']**3
    X['agesex'] = X['age']*X['sex']
    X['age2sex'] = X['age2']*X['sex']
        
    agesex = ['age','sex','agesex','age2','age2sex']
       
    X['lnage'] = np.log(X['age'])
    X['lnage2'] = X['lnage']**2
    
    X['CABG_PCI'] = ~pn_info.no_CABG_PCI_before
    X['CAD'] = ~pn_info.no_CAD_before
    X['MI'] = ~pn_info.no_MI_before
    X['PCE'] = ~pn_info.no_PCEend_before
    X['Stroke'] = ~pn_info.no_stroke_before
    X['Asthma'] = pn_info.Asthma
    X['cancer'] = pn_info.cancer
    X['Stroke'] = ~pn_info.no_stroke_before
    
    X['HDL'] = pn_info.HDL
    X['TC'] = pn_info.TC
    X['SBP'] = pn_info.SBP
    
    X['ApoB'] = np.log(pn_info['ApoB'])
    X['GDF15'] = np.log(pn_info['GDF15'])
    X['Smoker'] = pn_info['Smoker'].astype(int).values
    X['diabetes'] = pn_info['T2D'].astype(int).values
    X['HTN_treated'] = pn_info[['HTN_treated']].astype(int).values

#     X['statin'] = pn_info['statin'].astype(int).values
    X['statin'] = pn_info['statin_estimate_unsure'].astype(int).values
 
    X['bmi'] = pn_info['bmi']

    no_bmi = (X['bmi'].isna()); print(no_bmi.sum())
    no_bmi_ind = X[no_bmi].index
    X.loc[I_train.intersection(no_bmi_ind),'bmi'] = X.loc[I_train].bmi.mean()
    X['bmi2'] = X['bmi']*X['bmi']
    
    X['Platelets'] = pn_info['Platelets']
    no_p = (X['Platelets'].isna()); print(no_p.sum())
    no_p_ind = X[no_p].index
    X.loc[I_train.intersection(no_p_ind),'Platelets'] = X.loc[I_train].Platelets.mean()
    X['Platelets2'] = X['Platelets']*X['Platelets']
    
    X['Creatinine'] = pn_info['Creatinine']
    no_p = (X['Creatinine'].isna()); print(no_p.sum())
    no_p_ind = X[no_p].index
    X.loc[I_train.intersection(no_p_ind),'Creatinine'] = X.loc[I_train].Creatinine.mean()
    
    X['Triglycerides'] = pn_info['Triglycerides']
    no_p = (X['Triglycerides'].isna()); print(no_p.sum())
    no_p_ind = X[no_p].index
    X.loc[I_train.intersection(no_p_ind),'Triglycerides'] = X.loc[I_train].Triglycerides.mean()
    
    

    X['bmiage'] = X['bmi']*X['age']
    X['bmisex'] = X['bmi']*X['sex']
    X['bmi2age'] = X['bmi2']*X['age']
    X['bmi2sex'] = X['bmi2']*X['sex']
    X['ApoBage']  = X['ApoB']*X['age']
    X['Smokerage'] = X['Smoker']*X['age']
    X['diabetesage'] = X['diabetes']*X['age'] 
    X['statinage'] = X['statin']*X['age']
    X['CADage'] = X['CAD']*X['age']
    X['MIage'] = X['MI'] * X['age']
    X['HTN_treatedage'] =  X['age']*X['HTN_treated']    
    X['cancerage'] = X['age']*X['cancer']
    
    X['Plateletsage'] = X['Platelets']*X['age']
    X['Creatinineage'] = X['Creatinine']*X['age']
    X['Triglyceridesage'] = X['Triglycerides']*X['age']
    X['Platelets2age'] = X['Platelets2']*X['age']
    
    X['ApoBsex']  = X['ApoB']*X['sex']
    X['Smokersex'] = X['Smoker']*X['sex']
    X['diabetessex'] = X['diabetes']*X['sex'] 
    X['statinsex'] = X['statin']*X['sex']
    X['CADsex'] = X['CAD']*X['sex']
    X['MIsex'] = X['MI'] * X['sex']
    X['HTN_treatedsex'] =  X['sex']*X['HTN_treated']    
    X['cancersex'] = X['sex']*X['cancer']
    
    X['Plateletssex'] = X['Platelets']*X['sex']
    X['Creatininesex'] = X['Creatinine']*X['sex']
    X['Triglyceridessex'] = X['Triglycerides']*X['sex']  
    X['Platelets2sex'] = X['Platelets2']*X['sex']
    
    X['ApoBstatin']  = X['ApoB']*X['statin']
    
    X = X.join(pd.get_dummies(pn_info['agebin'],drop_first = True,prefix='age'))
    X['ageage2'] = X['age']*X['age_2.0']
    X['ageage3'] = X['age']*X['age_3.0']
    X['ageage4'] = X['age']*X['age_4.0']
    
    agebins = ['age_2.0','age_3.0','age_4.0', 'ageage2','ageage3','ageage4']
    agebins60 = ['age_4.0','ageage4']
    agebins60sex = ['age_4.0sex','ageage4sex']
    agebinssex = [s+'sex' for s in agebins]
    X[agebinssex] = (X[agebins].transpose()*X['sex']).transpose()    
        
    
    PRS = ['nonHDL_prs', 'HT_prs', 'CAD_prs', 'Cancer_prs', 'Stroke2_prs', 'alz_Jansen',
       'pgc_adhd_2017', 'PD_Nalls_2018', 'edu_160125', 'dep_2018', 'bpd_2018',
       'giant_bmi', 'schizo_clozuk', 'iq_2018', 'ipsych_pgc_aut_2017',
       'pgc_Anorexia_2019']
    X[PRS] = pn_info[PRS]
    X[qt_col] = pn_info[qt_col]
    
#     for c in qt_col[1:100]: 
#         X[c+'2'] = X[c]**2
#         X[c+'age'] = X[c]*X['age']
#         X[c+'sex'] = X[c]*X['sex']

    for c in extra_feat: 
        X[c+'2'] = X[c]**2
        X[c+'age'] = X[c]*X['age']
        X[c+'sex'] = X[c]*X['sex']
    
    trad = ['ApoB','Smoker','diabetes','HTN_treated','statin','CAD','MI','bmi','cancer']
    tradage = ['ApoBage','Smokerage','diabetesage','CADage','MIage','HTN_treatedage','bmiage','statinage','cancerage']
    tradsex = ['ApoBsex','Smokersex','diabetessex','CADsex','MIsex','HTN_treatedsex','bmisex','statinsex','cancersex']
    
    tradcoxR = ['Smoker','Smokersex','diabetes','diabetesage','HTN_treated','HTN_treatedage','MI','MIage','CAD','bmi','bmiage','statin','statinage']
    tradblood = ['Platelets','Platelets2','Creatinine','Triglycerides']
    tradbloodextra = ['Plateletsage','Creatinineage','Triglyceridesage','Platelets2age','Plateletssex','Creatininesex','Triglyceridessex','Platelets2sex']

    tradextralog = ['Smokersex','diabetessex','CADsex','CADage','MIage','HTN_treatedage','bmiage','statinage','bmi2','bmi2age','ApoBstatin','cancerage']  
    
    org_baseline = trad+tradextralog
    new_baseline = ['ApoB','Smoker','diabetes','HTN_treated','statin','CAD','MI','bmi','bmi2','Stroke','Smokersex',
             'diabetessex','CADsex','CADage','MIage','HTN_treatedage','bmiage','statinage','bmi2age','ApoBstatin',
            'cancer','cancerage','cancersex']
    
    X_train = X.loc[I_train]
    X_test = X.loc[I_test]

    train_mean = X_train.mean()
    train_std = X_train.std()

    X_train = (X_train-train_mean)/train_std
    X_test = (X_test-train_mean)/train_std

        ## For survival analysis    
    X_train['event'] = use_event[I_train]
    X_test['event'] = use_event[I_test]
   
    X_train['time_to_event'] =  time_to_event[I_train]
    X_test['time_to_event'] = time_to_event[I_test]
    
    batch_var = pd.get_dummies(pn_info.loc[I_train,'batch'],drop_first = True).columns
    X_train[batch_var] = pd.get_dummies(pn_info.loc[I_train,'batch'],drop_first = True)


    y_train = y[k][I_train]
    y_test = y[k][I_test]

#         ycsurv_train = pd.DataFrame()
#         ycsurv_train['event'] = y_train
#         ycsurv_train['time_to_event'] = np.where(tte_train<= (k+1),tte_train,k+1)

    X_train['y'] = y_train
    X_test['y'] = y_test

    try:

        file = open(folder+feat_folder+"{}_{}_features.pkl".format(endpoint,dataset),'rb')
        features_dict = pickle.load(file)           

        boruta = sorted(features_dict['{}_boruta_y{}'.format(dataset,k)])
        print('features_loaded')

    except Exception as e:
        print(e)
        print('No features')

    try: 
        file = open(folder+pred_folder+"{}_{}_predict_trad_cv.pkl".format(endpoint,dataset),'rb')
        pred_dict_cv = pickle.load(file)
        
#         file = open(folder+pred_folder+"{}_{}_predict_trad_2_cv.pkl".format(endpoint,dataset),'rb')
#         pred_dict_cv_2 = pickle.load(file)
        
#         file = open(folder+pred_folder+"{}_{}_predict_trad_cv_3.pkl".format(endpoint,dataset),'rb')
#         pred_dict_cv_3 = pickle.load(file)
        
#         pred_dict_cv = {**pred_dict_cv, **pred_dict_cv_2, **pred_dict_cv_3}  
    except:
        pred_dict_cv = {}

In [ ]:
pred_dict_cv.keys()
# pred_dict_cv_2.keys()

In [ ]:
# k=4
# y_train = y[k][I_train]

In [ ]:
feat = ['age','sex','agesex','age2','age2sex']
# feat.extend(agebins)
# feat.extend(agebins60)
logit = sm.Logit(y_train, sm.add_constant(X_train[feat])).fit()
logit.summary()

In [ ]:
count = 0 
sign_count = 0
for c in w1y_qt_col:
    feat = ['age','sex','agesex','age2','age2sex']
    feat.append(c)
    x_train = X_train[~X_train[c].isna()]
    if x_train.shape[0]<10000:
        continue
    logit = sm.Logit(y_train[x_train.index], sm.add_constant(x_train[feat])).fit()
    display(logit.summary())
    count = count +1 
    pval = logit.pvalues[c]
    print(pval)
    if pval < 0.05:
        sign_count = sign_count+1
    
    

In [ ]:
print(count)
print(sign_count)
      

In [ ]:
pvalues = []
pvalues_columns = []
count = 0 
sign_count = 0
for c in extra_feat2:
# for c in w1y_qt_col:
    feat = ['age','sex','agesex','age2','age2sex']
    feat.append(c)
    x_train = X_train[~X_train[c].isna()]
    if x_train.shape[0]<5000:
        continue
    logit = sm.Logit(y_train[x_train.index], sm.add_constant(x_train[feat])).fit()
    display(logit.summary())
    count = count +1 
    pval = logit.pvalues[c]
    print(pval)
    pvalues.append(pval)
    pvalues_columns.append(c)
    if pval < 0.05:
        sign_count = sign_count+1
    
    try:
        feat = []
        feat.extend(agesex)
#         feat.extend(trad)
#         feat.extend(tradextralog)
        feat.extend(new_baseline)
        out = pf.predict_cv(feat=feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lr',feat_sel_type = None)
        pred_dict_cv['{}_base2_{}_lr_y{}'.format(dataset,c,k)] = out
  
        feat.append(c)
        out = pf.predict_cv(feat=feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lr',feat_sel_type = None)
        pred_dict_cv['{}_base2plus{}_lr_y{}'.format(dataset,c,k)] = out
        
#         GDF15 = ['GDF15']
#         GDF15.extend(feat)
#         out = pf.predict_cv(feat=GDF15,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lr',feat_sel_type = None)
#         pred_dict_cv['{}_baseGDF15_{}_lr_y{}'.format(dataset,c,k)] = out
        
#         protein_feat = []
#         protein_feat.extend(boruta)
#         protein_feat.extend(feat)
#         out = pf.predict_cv(feat=protein_feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lrl1',feat_sel_type = None)
#         pred_dict_cv['{}_baseprotein_{}_lr_y{}'.format(dataset,c,k)] = out

#         protein_feat = []
#         protein_feat.extend(agesex)
#         protein_feat.extend(boruta)
# #         protein_feat.extend(feat)
#         out = pf.predict_cv(feat=protein_feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lrl1',feat_sel_type = None)
#         pred_dict_cv['{}_agesexprotein_{}_lr_y{}'.format(dataset,c,k)] = out
        
        
#         feat = []
#         feat.extend(agesex)
#         feat.extend(trad)
#         feat.extend(tradextralog)
#         feat.extend([c,c+'2',c+'age',c+'sex'])
#         out = pf.predict_cv(feat=feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lrl2',feat_sel_type = None)
#         pred_dict_cv['{}_{}_extra_lr_y{}'.format(dataset,c,k)] = out
        
#         feat = []
#         feat.extend(agesex)
#         feat.extend(trad)
#         feat.extend(tradextralog)
#         transformer = QuantileTransformer(n_quantiles=50000, output_distribution = 'normal',random_state=10)
#         transformer.fit(np.array(x_train[c]).reshape(-1,1))
#         x_train[c+'_qt'] = transformer.transform(np.array(x_train[c]).reshape(-1,1))
#         feat.append(c+'_qt')
#         out = pf.predict_cv(feat=feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lr',feat_sel_type = None)
#         pred_dict_cv['{}_{}_qt_lr_y{}'.format(dataset,c,k)] = out        

        f = open(folder+pred_folder+"{}_{}_predict_trad_cv.pkl".format(endpoint,dataset),"wb")
        pickle.dump(pred_dict_cv,f)
        f.close()
    except Exception as e:
        print(e)
        print('Fail')
pvalues = pd.DataFrame(pvalues,index=pvalues_columns)

In [ ]:
#         f = open(folder+pred_folder+"{}_{}_predict_trad_cv.pkl".format(endpoint,dataset),"wb")
#         pickle.dump(pred_dict_cv,f)
#         f.close()

In [ ]:
pvalues

In [ ]:
multicols = {'Bloodcells': ['Hematocrit_w1y', 'Hemoglobin_w1y','MCH_w1y','MCHC_w1y','MCV_w1y','Platelets_w1y','WBC_w1y','RDW_w1y'],#(RBC_w1y)
             'Suggested': ['SBP','Glucose_w1y','Triglycerides_w1y'],
             'Top3': ['ALP_w1y','ESR_w1y','RDW_w1y'],
             'Top3+4High':['ALP_w1y','ESR_w1y','RDW_w1y','Fibrosis-4_w1y','Hematocrit_w1y', 'Hemoglobin_w1y','RBC_w1y'],
             'Likely_candidates': ['SBP','Glucose_w1y','Triglycerides_w1y','HDL','TC','Creatinine_w1y'],
             'CV_risk': ['SBP','HDL','TC']}

included_names = ['Bloodcells','Likely3','Top3','High7','Likely6','CV risk']
#              'reviewer_nw1y': ['SBP','Glucose','Triglycerides'],

In [ ]:
w1y_qt_col

In [ ]:
for c,val in multicols.items():
# for c in w1y_qt_col:    
    print(c)
    print(val)
    x_train = X_train[(~X_train[val].isna()).all(axis=1)]
    print(x_train.shape)
#     if x_train.shape[0]<5000:
#         continue
    try:
        feat = []
        feat.extend(agesex)
#         feat.extend(trad)
#         feat.extend(tradextralog)
        feat.extend(new_baseline)
        out = pf.predict_cv(feat=feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lrl2',feat_sel_type = None)
        pred_dict_cv['{}_base2_{}_lr_y{}'.format(dataset,c,k)] = out
        
        feat.extend(val)
        out = pf.predict_cv(feat=feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lrl2',feat_sel_type = None)
        pred_dict_cv['{}_base2plus{}_lr_y{}'.format(dataset,c,k)] = out
        
#         protein_feat = []
#         protein_feat.extend(boruta)
#         protein_feat.extend(feat)
#         out = pf.predict_cv(feat=protein_feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lrl1',feat_sel_type = None)
#         pred_dict_cv['{}_baseprotein_{}_lr_y{}'.format(dataset,c,k)] = out
        
#         protein_feat = []
#         protein_feat.extend(boruta)
#         protein_feat.extend(agesex)
#         out = pf.predict_cv(feat=protein_feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lrl1',feat_sel_type = None)
#         pred_dict_cv['{}_agesexprotein_{}_lr_y{}'.format(dataset,c,k)] = out
        
        
#         transformer = QuantileTransformer(n_quantiles=50000, output_distribution = 'normal',random_state=10)
#         transformer.fit(np.array(x_train[c]).reshape(-1,1))
#         x_train[c+'_qt'] = transformer.transform(np.array(x_train[c]).reshape(-1,1))
#         feat.append(c+'_qt')
#         out = pf.predict_cv(feat=feat,kf=kf,X=x_train,y=y[k][x_train.index],model_type = 'lr',feat_sel_type = None)
#         pred_dict_cv['{}_{}_qt_lr_y{}'.format(dataset,c,k)] = out        

        f = open(folder+pred_folder+"{}_{}_predict_trad_cv.pkl".format(endpoint,dataset),"wb")
        pickle.dump(pred_dict_cv,f)
        f.close()
    except Exception as e:
        print(e)
        print('Fail')

In [ ]:
# f = open(folder+pred_folder+"{}_{}_predict_trad_cv.pkl".format(endpoint,dataset),"wb")
# pickle.dump(pred_dict_cv,f)
# f.close()

In [ ]:
inc_qt = []
for c in w1y_qt_col:
    x_train = X_train[~X_train[c].isna()]
    if x_train.shape[0]<5000:
        continue
    inc_qt.append(c)
len(inc_qt)

In [ ]:
single_feat=True

In [ ]:
### Featutes in the big picture
all_extra_feat = extra_feat2[1:]+list(w1y_qt_col)
all_extra_feat.remove('HDL_w1y')
all_extra_feat.remove('Total_cholesterol_w1y')

In [ ]:
### Select which one of the plots to make

all_extra_feat_N = []
all_extra_feat_case = []

### Addin one feature at a time
if single_feat:
    included_names = []
    for c in all_extra_feat:
        x_train = X_train[~X_train[c].isna()]
        if x_train.shape[0]<5000:
            continue
        all_extra_feat_N.append(x_train.shape[0])
        all_extra_feat_case.append(y[k][x_train.index].sum())
        if 'w1y' in c:
            c = c.replace('_w1y','')
        included_names.append(c)
    
### Add combination of features
else:    
    for c, val in multicols.items():
        x_train = X_train[(~X_train[val].isna()).all(axis=1)]
        all_extra_feat_N.append(x_train.shape[0])
        all_extra_feat_case.append(y[k][x_train.index].sum())

In [ ]:
file = open(folder+org_folder+"{}_{}_predict_trad_cv.pkl".format(endpoint,dataset),'rb')
pred_dict_org_cv = pickle.load(file)
list(pred_dict_org_cv.keys())

for key, value in pred_dict_org_cv.items():
    if 'agesex' in key:
        print(key)
        pred_dict_cv[key] = value


In [ ]:
k=4
keep_samples = keep_samples_dict[dataset]

I_train = I_train_main.intersection(keep_samples)
I_test = I_test_main.intersection(keep_samples)
tte_train = time_to_event[I_train]
score_dict = {}
base_score_dict = {}
protein_score_dict = {}
include_protein = True
# try: 
#     file = open(folder+pred_folder+"{}_{}_predict_trad_cv.pkl".format(endpoint,dataset),'rb')
#     pred_dict_cv = pickle.load(file)

# except:
#     pred_dict_cv = {}

keys = pred_dict_cv.keys()
# print(keys)


included_qt = []
bins = [0,0.075,1]
# for c, val in multicols.items():
# for c in w1y_qt_col:
for c in all_extra_feat:
# for c in extra_feat:
# for c in pvalues_columns:

    if single_feat:
        x_train_index = X_train.index[~X_train[c].isna()]
        if len(x_train_index)<5000:
            continue
    else:
        x_train_index = X_train.index[(~X_train[val].isna()).all(axis=1)]

    included_qt.append(c)
#     key = '{}_{}_lr_y{}'.format(dataset,c,k)
    key = '{}_base2plus{}_lr_y{}'.format(dataset,c,k)

    out = []
    base_out = []
    protein_out = []
    fold_num = 0
    for train_index, test_index in kf.split(x_train_index):
        fold = pred_dict_cv[key][2][fold_num]
        numf = pred_dict_cv[key][0][fold_num]
        y_test = y[k][x_train_index].iloc[test_index]
#         tte_test = tte_train.iloc[test_index]
#         base_fold = pred_dict_cv['{}_agesex_lr_y{}'.format(dataset,k)][2][fold_num]        
#         base_fold = pred_dict_cv['{}_base_{}_lr_y{}'.format(dataset,c,k)][2][fold_num] 
        base_fold = pred_dict_cv['{}_base2_{}_lr_y{}'.format(dataset,c,k)][2][fold_num] 
        if include_protein: 
            protein_fold = pred_dict_cv['{}_agesexprotein_{}_lr_y{}'.format(dataset,c,k)][2][fold_num]  
#         y_cens_test = y_cens[k][I_train].iloc[test_index]
#         print(key)
        auc =[]
        base_auc = []
        protein_auc = []
        for i,case in enumerate(fold):
            auc.append(len(numf[i]))
#             auc.extend(calculate_metrics(np.array(case).reshape(-1)[~y_cens_test],np.array(base_fold[0]).reshape(-1)[~y_cens_test],y_test[~y_cens_test],bins=bins))
            auc.extend(calculate_metrics(np.array(case).reshape(-1),np.array(base_fold[0]).reshape(-1),y_test,bins=bins))
    
            base_auc.append(len(numf[i])-1)
            base_auc.extend(calculate_metrics(np.array(base_fold[0]).reshape(-1),np.array(base_fold[0]).reshape(-1),y_test,bins=bins))
            if include_protein:
                protein_auc.append(len(numf[i])-1)
                protein_auc.extend(calculate_metrics(np.array(protein_fold[0]).reshape(-1),np.array(base_fold[0]).reshape(-1),y_test,bins=bins))

        
        out.append(auc)
        base_out.append(base_auc)
        if include_protein:
            protein_out.append(protein_auc)
        fold_num = fold_num+1
    score_dict[key] = out
    base_score_dict[key] = base_out
    if include_protein:
        protein_score_dict[key] = protein_out
#             print(len(y_test), len(case))
# f = open(folder+pred_folder+"{}_{}_score_cv.pkl".format(endpoint,dataset),"wb")
# pickle.dump(score_dict,f)
# f.close()

In [ ]:
VERY_SMALL_SIZE = 12
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=VERY_SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE) 

In [ ]:
keys = score_dict.keys()
if single_feat:
    fig = plt.figure(figsize=[15,6])
else:
    fig = plt.figure(figsize=[6,6])
ax = fig.add_subplot(1,1,1)
ax2 = ax.twinx()
# fig = plt.figure(figsize=[12,6])
for i,key in enumerate(keys):
    out=score_dict[key]
    base_out = base_score_dict[key]
    protein_out = protein_score_dict[key]
#     plt.scatter(np.array(out).mean(axis=0)[:,0],np.array(out).mean(axis=0)[:,1])
    ax.scatter(i,np.array(out).mean(axis=0)[1]-np.array(base_out).mean(axis=0)[1],color='blue')
    ax.scatter(i,np.array(protein_out).mean(axis=0)[1]-np.array(base_out).mean(axis=0)[1],color='red')
    ax2.bar(i,all_extra_feat_N[i],color = 'black',alpha=0.2)
    ax2.bar(i,all_extra_feat_case[i],color = 'red',alpha=0.2)
ax.grid()
ax.set_xticks(ticks = range(len(included_qt)))
# ax.set_xticklabels(labels=included_qt, rotation = 90)
ax.set_xticklabels(labels=included_names, rotation = 90)
ax.legend(['Baseline+feature','Age+sex+proteins'])
ax2.legend(['N','Cases'],loc='upper left',bbox_to_anchor=(1.0, 1.0))
ax.set_xlabel('Features')
ax.set_ylabel('Model AUC - Baseline AUC')
ax2.set_ylabel('Number of samples')

plt.title('Ten-fold CV, 5-year mortality risk')
# lgd = plt.legend(keys,loc='center left', bbox_to_anchor=(1.0, 0.5))
if single_feat:
    plt.savefig(folder+pred_folder+"{}_{}_AUC_diff_wN_y{}_base2_extra_all.png".format(endpoint,dataset,k), bbox_inches='tight')
else:
    plt.savefig(folder+pred_folder+"{}_{}_AUC_diff_wN_y{}_base2_extra_multicol.png".format(endpoint,dataset,k), bbox_inches='tight')
plt.show()

In [ ]:
# keys = pred_dict_cv.keys()
keys = score_dict.keys()
# fig = plt.figure(figsize=[6,6])
for i,key in enumerate(keys):
    out=score_dict[key]
    base_out = base_score_dict[key]
#     plt.scatter(np.array(out).mean(axis=0)[:,0],np.array(out).mean(axis=0)[:,1])
    plt.scatter(i,np.array(out).mean(axis=0)[1]-np.array(base_out).mean(axis=0)[1])

lgd = plt.legend(keys,loc='center left', bbox_to_anchor=(1.0, 0.5))
# plt.savefig(folder+"Predictions_cv/{}_{}_AUC_y{}.png".format(endpoint,dataset,k), bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

# fig = plt.figure(figsize=[12,6])
for i,key in enumerate(keys):
    out=score_dict[key]
    plt.scatter(i,np.array(out).mean(axis=0)[1])
# plt.legend(keys)
lgd = plt.legend(keys,loc='center left', bbox_to_anchor=(1.0, 0.5))

# plt.savefig(folder+pred_folder+"{}_{}_logloss_y{}.png".format(endpoint,dataset,k),bbox_extra_artists=(lgd,), bbox_inches='tight')
plt.show()

In [ ]:
keys = score_dict.keys()
print(keys)
fig = plt.figure(figsize=[15,6])
# fig = plt.figure(figsize=[12,6])
for i,key in enumerate(keys):
    out=score_dict[key]
    base_out = base_score_dict[key]
#     protein_out = protein_score_dict[key]
#     plt.scatter(np.array(out).mean(axis=0)[:,0],np.array(out).mean(axis=0)[:,1])
    plt.scatter(i,np.array(out).mean(axis=0)[1]-np.array(base_out).mean(axis=0)[1],color='blue')
    plt.scatter(i,np.array(protein_out).mean(axis=0)[1]-np.array(base_out).mean(axis=0)[1],color='red')
plt.grid()
plt.xticks(ticks = range(len(included_qt)),labels=included_qt, rotation = 90)
plt.legend(['Baseline+risk factor','Age+sex+proteins'])
plt.xlabel('Dataset/ Risk factor')
plt.ylabel('Model AUC - baseline AUC')
plt.title('Ten-fold CV, 5-year mortality risk')
# lgd = plt.legend(keys,loc='center left', bbox_to_anchor=(1.0, 0.5))
# plt.savefig(folder+pred_folder+"{}_{}_AUC_diff_y{}_extra_all.png".format(endpoint,dataset,k), bbox_inches='tight')
# plt.show()

In [ ]:
# print((~X_train[pvalues_columns].isna()).all(axis=1).sum())
# print((~X_train[important_qt].isna()).all(axis=1).sum())
print((~X_train[['Hematocrit_w1y', 'Hemoglobin_w1y','RDW_w1y']].isna()).all(axis=1).sum())
print((~X_train[['Creatinine_w1y','SBP','Glucose_w1y','HDL','Platelets_w1y','Triglycerides_w1y','ESR_w1y']].isna()).all(axis=1).sum())
print((~X_train[['Creatinine_w1y','Platelets_w1y']].isna()).all(axis=1).sum())
print((~X_train[['Creatinine_w1y','Platelets_w1y','ESR_w1y']].isna()).all(axis=1).sum())
print((~X_train[['Creatinine_w1y','Platelets_w1y','ESR_w1y','Hematocrit_w1y', 'Hemoglobin_w1y','RDW_w1y']].isna()).all(axis=1).sum())
print((~X_train[['Creatinine_w1y','Platelets_w1y','Hematocrit_w1y', 'Hemoglobin_w1y','RDW_w1y']].isna()).all(axis=1).sum())
print((~X_train[['Creatinine_w1y','SBP','Glucose_w1y','HDL','Triglycerides_w1y']].isna()).all(axis=1).sum())
print((~X_train[['SBP','Glucose_w1y','Triglycerides_w1y']].isna()).all(axis=1).sum())
print((~X_train[['SBP','Glucose_w1y']].isna()).all(axis=1).sum())
print((~X_train[['SBP','Glucose','Triglycerides']].isna()).all(axis=1).sum())
print((~X_train[['Hematocrit_w1y', 'Hemoglobin_w1y','MCH_w1y','MCHC_w1y','MCV_w1y','Platelets_w1y','WBC_w1y','RDW_w1y']].isna()).all(axis=1).sum()) ### All bood cell counts
print((~X_train[['ALP_w1y','ESR_w1y','RDW_w1y']].isna()).all(axis=1).sum())
print((~X_train[['ALP_w1y','ESR_w1y','RDW_w1y','Fibrosis-4_w1y']].isna()).all(axis=1).sum())
print((~X_train[['ALP_w1y','ESR_w1y','RDW_w1y','Fibrosis-4_w1y','Hematocrit_w1y', 'Hemoglobin_w1y','RBC_w1y']].isna()).all(axis=1).sum())
print((~X_train[['Creatinine_w1y','Glucose_w1y','Platelets_w1y','Triglycerides_w1y','ESR_w1y']].isna()).all(axis=1).sum())
print((~X_train[['SBP','Glucose_w1y','Triglycerides_w1y','HDL','TC','Creatinine_w1y']].isna()).all(axis=1).sum())

In [ ]:
['Hematocrit_w1y', 'Hemoglobin_w1y','RDW_w1y']
['Creatinine_w1y','SBP','Glucose_w1y','non_HDL','Platelets_w1y','Triglycerides_w1y']
important_qt

In [ ]:
(~X_train[pvalues_columns].isna()).sum()